## Task1

-> Objetivo: Dada uma consulta especificada pelo usuário e uma lista de produtos correspondentes, o objetivo desta tarefa é classificar os produtos para que os produtos relevantes sejam classificados acima dos não relevantes.

A entrada para esta tarefa será uma lista de consultas com seus identificadores. O sistema terá que gerar um arquivo CSV onde o **query_id** estará na primeira coluna e o **product_id** na segunda coluna, onde para cada **query_id** primeira linha será o produto mais relevante e a última linha o produto menos relevante

In [12]:
!pip install lightgbm
!pip install annoy
!pip install faiss-cpu
!pip install -U sentence-transformers
!pip install nltk

Requirement already up-to-date: sentence-transformers in /home/erick.correia/.local/lib/python3.8/site-packages (2.2.2)


In [45]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import lightgbm as lgb
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder 

import lightgbm as lgb
import faiss
from annoy import AnnoyIndex

In [44]:
train_df = pd.read_csv('dataset/train-v0.3.csv')

In [46]:
print("shape train data = " , train_df.shape)

shape train data =  (781744, 5)


In [4]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781744 entries, 0 to 781743
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   query_id      781744 non-null  int64 
 1   query         781744 non-null  object
 2   query_locale  781744 non-null  object
 3   product_id    781744 non-null  object
 4   esci_label    781744 non-null  object
dtypes: int64(1), object(4)
memory usage: 29.8+ MB


,query_id,query,query_locale,product_id,esci_label
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact
4,0,# 2 pencils not sharpened,us,B001B097KC,exact


In [5]:
train_df.isnull().sum()

query_id        0
query           0
query_locale    0
product_id      0
esci_label      0
dtype: int64

**Removendo StopWords / lowcase / caracteres especiais** 

In [6]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:
text = train_df['query']
text = text[:419730]

stop_words = stopwords.words('english')

In [8]:
#print(text)
#text_tokens = word_tokenize(text)
text= text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#remove lowcase
text = text.str.lower()
#revome caracteres especiais 
text = text.str.replace('[#,@,&,1-9]','')
print(text)

0            pencils sharpened
1            pencils sharpened
2            pencils sharpened
3            pencils sharpened
4            pencils sharpened
                  ...         
419725    zephyr polishing kit
419726    zephyr polishing kit
419727    zephyr polishing kit
419728    zephyr polishing kit
419729    zephyr polishing kit
Name: query, Length: 419730, dtype: object


/tmp/ipykernel_15048/165082349.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('[#,@,&,1-9]','')


**Criando vetores**

In [9]:
import requests
from io import StringIO
#import pandas as pd

In [10]:
#sentences = train_df['query'].tolist()
sentences = text.tolist()

In [11]:
# remove duplicates and NaN
#sentences = [word for word in list(set(sentences)) if type(word) is str]
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [12]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens') 
#paraphrase-multilingual-MiniLM-L12-v2

/home/erick.correia/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(20815, 768)

## IndexFlatL2



In [14]:
import faiss

In [16]:
size = sentence_embeddings.shape[1]
size

768

In [17]:
index = faiss.IndexFlatL2(size)

In [18]:
index.is_trained

True

In [19]:
index.add(sentence_embeddings)

In [20]:
index.ntotal

20815

In [21]:
#Teste
k = 4
xq = model.encode(["envelopes without security tint"])

In [22]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[19805  4829   978 14630]]
CPU times: user 19.1 ms, sys: 35 µs, total: 19.1 ms
Wall time: 17.6 ms


In [27]:
train_df['query'].iloc[[19805, 4829, 978, 14630]]

19805                6xl knee sleeve with straps
4829     1 ply disposable mask without nose clip
978              'module' object is not callable
14630                            2x3 white board
Name: query, dtype: object

In [25]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, size))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [28]:
vecs.shape

(4, 768)

In [29]:
vecs[0][:100]

array([ 0.43119857,  0.91011864,  0.4051747 , -0.06226606,  0.09338088,
       -0.20125297,  1.1626184 , -0.08322101, -0.36759871,  0.20486532,
       -0.93038219, -0.61459619,  0.4171924 ,  0.51536101,  0.88794357,
        0.45378926, -0.57322139,  0.27390629,  0.29525098, -0.19188608,
       -0.24476814,  0.57090127,  0.04908442, -1.19785392, -0.54007924,
       -0.35953718,  0.39977959, -0.30137578, -1.15571201,  0.45229897,
        0.232692  ,  0.55188435,  1.23124671,  0.08548499,  1.41238093,
       -0.40255302, -0.1310015 , -1.07996678, -0.15451427, -0.60550433,
       -0.06073511,  0.32497635,  0.32338566,  0.35931736,  0.13488162,
       -0.05715447,  1.5942353 ,  0.02342471, -0.78752214,  0.07201277,
        0.56621379, -0.50664479,  1.9796375 ,  1.1744287 , -0.15058279,
       -0.67751205,  0.21461996, -0.77990431,  0.93161666, -0.23863807,
        0.13793547,  0.04795064, -0.26176697,  0.12465028, -0.2205309 ,
       -0.32104367,  0.07732798,  0.5417518 , -0.53027701, -1.18

# implementando um indice plano faiss

In [31]:
index_x = faiss.IndexFlatIP(size)

In [32]:
index_x.add(sentence_embeddings)

In [33]:
xq = model.encode(["car paint"])

In [34]:
#Teste
k = 10
D, I = index_x.search(xq, k)
print(I)

[[10365 12349  7469  7340 11175    50  2151 10218  2665 12706]]


In [35]:
train_df['query'].iloc[[10365, 12349, 7469, 7340]]

10365          12 pc pot and pan set
12349                1u blank vented
7469       10 oz tumbler without lid
7340     10 lb weights dumbbells set
Name: query, dtype: object

In [37]:
nbits = size*4  # resolution of bucketed vectors
# initialize index and add vectors
index_e = faiss.IndexLSH(size, nbits)
index_e.add(sentence_embeddings)
# and search
D, I = index_e.search(xq, k)
print(I)

[[12349  7469  9072 10365  4282  2151  7340    50 11175  2665]]


In [39]:
train_df['query'].iloc[[12349, 7469, 9072, 10365, 4282, 2151, 7340, 50, 11175, 2665 ]]

12349                                   1u blank vented
7469                          10 oz tumbler without lid
9072                   10x 13 picture without the frame
10365                             12 pc pot and pan set
4282     1 inflatable light up pool ball without remote
2151                           00 gauge funny christmas
7340                        10 lb weights dumbbells set
50                                           # mom life
11175                                   13.5 shoes boys
2665      08 bmw x5 a/c air compressor without adaptive
Name: query, dtype: object

In [41]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index_h = faiss.IndexHNSWFlat(size, M)
# set efConstruction and efSearch parameters
index_h.hnsw.efConstruction = ef_construction
index_h.hnsw.efSearch = ef_search
# add data to index
index_h.add(xq)

# search as usual
D, I = index.search(xq, k)
print(I)

[[12349  7469 10365  9072  2665  7340 11175 10218  2151  1005]]
